# Bayesian_Hyperparameter_Optimization
* https://www.youtube.com/watch?v=1f4psgAcefU&list=PLjy4p-07OYzuy_lHcRW8lPTLPTTOmUpmi&index=40
* https://github.com/jeffheaton/app_deep_learning/blob/main/t81_558_class_08_4_bayesian_hyperparameter_opt.ipynb

Bayesian Hyperparameter Optimization is a method of finding hyperparameters more efficiently than a grid search. Because each candidate set of hyperparameters requires a retraining of the neural network, it is best to keep the number of candidate sets to a minimum. Bayesian Hyperparamter Optimization achieves this by training a model to predict good candidate sets of hyperparameters.
* https://github.com/bayesian-optimization/BayesianOptimization

In [1]:
# Startup Google CoLab
try:
    import google.colab
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# Nicely formatted time string


def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Make use of a GPU or MPS (Apple) if one is available.  (see module 3.2)
import torch
has_mps = torch.backends.mps.is_built()
device = "mps" if has_mps else "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Note: using Google CoLab
Using device: cuda


In [2]:
from scipy.stats import zscore
import pandas as pd
import logging
import os
logging. disable(logging.WARNING)

# Read the data set
df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/jh-simple-dataset.csv",
    na_values=['NA', '?'])

df.head()

,id,job,area,income,aspect,subscriptions,dist_healthy,save_rate,dist_unhealthy,age,pop_dense,retail_dense,crime,product
0,1,vv,c,50876.0,13.100000,1,9.017895,35,11.738935,49,0.885827,0.492126,0.071100,b
1,2,kd,c,60369.0,18.625000,2,7.766643,59,6.805396,51,0.874016,0.342520,0.400809,c
2,3,pe,c,55126.0,34.766667,1,3.632069,6,13.671772,44,0.944882,0.724409,0.207723,b
3,4,11,c,51690.0,15.808333,1,5.372942,16,4.333286,50,0.889764,0.444882,0.361216,b
4,5,kl,d,28347.0,40.941667,3,3.822477,20,5.967121,38,0.744094,0.661417,0.068033,a


In [3]:
# Generate dummies for job
df = pd.concat(
    [df, pd.get_dummies(df['job'], prefix="job", dtype=int)], axis=1)
df.drop('job', axis=1, inplace=True)
df

,id,area,income,aspect,subscriptions,dist_healthy,save_rate,dist_unhealthy,age,pop_dense,...,job_pe,job_po,job_pq,job_pz,job_qp,job_qw,job_rn,job_sa,job_vv,job_zz
0,1,c,50876.0,13.100000,1,9.017895,35,11.738935,49,0.885827,...,0,0,0,0,0,0,0,0,1,0
1,2,c,60369.0,18.625000,2,7.766643,59,6.805396,51,0.874016,...,0,0,0,0,0,0,0,0,0,0
2,3,c,55126.0,34.766667,1,3.632069,6,13.671772,44,0.944882,...,1,0,0,0,0,0,0,0,0,0
3,4,c,51690.0,15.808333,1,5.372942,16,4.333286,50,0.889764,...,0,0,0,0,0,0,0,0,0,0
4,5,d,28347.0,40.941667,3,3.822477,20,5.967121,38,0.744094,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,c,51017.0,38.233333,1,5.454545,34,14.013489,41,0.881890,...,0,0,0,0,0,0,0,0,1,0
1996,1997,d,26576.0,33.358333,2,3.632069,20,8.380497,38,0.944882,...,0,0,0,0,0,0,0,0,0,0
1997,1998,d,28595.0,39.425000,3,7.168218,99,4.626950,36,0.759843,...,0,0,0,0,0,0,0,0,0,0
1998,1999,c,67949.0,5.733333,0,8.936292,26,3.281439,46,0.909449,...,0,0,0,0,1,0,0,0,0,0


In [11]:
df.columns

Index(['id', 'area', 'income', 'aspect', 'subscriptions', 'dist_healthy',
       'save_rate', 'dist_unhealthy', 'age', 'pop_dense', 'retail_dense',
       'crime', 'product', 'job_11', 'job_al', 'job_am', 'job_ax', 'job_bf',
       'job_by', 'job_cv', 'job_de', 'job_dz', 'job_e2', 'job_f8', 'job_gj',
       'job_gv', 'job_kd', 'job_ke', 'job_kl', 'job_kp', 'job_ks', 'job_kw',
       'job_mm', 'job_nb', 'job_nn', 'job_ob', 'job_pe', 'job_po', 'job_pq',
       'job_pz', 'job_qp', 'job_qw', 'job_rn', 'job_sa', 'job_vv', 'job_zz'],
      dtype='object')

In [4]:
# Generate dummies for area
df = pd.concat(
    [df, pd.get_dummies(df['area'], prefix="area", dtype=int)], axis=1)
df.drop('area', axis=1, inplace=True)

# Missing values for income
med = df['income'].median()
df['income'] = df['income'].fillna(med)

# Standardize ranges
df['income'] = zscore(df['income'])
df['aspect'] = zscore(df['aspect'])
df['save_rate'] = zscore(df['save_rate'])
df['age'] = zscore(df['age'])
df['subscriptions'] = zscore(df['subscriptions'])

df.head()

,id,income,aspect,subscriptions,dist_healthy,save_rate,dist_unhealthy,age,pop_dense,retail_dense,...,job_qp,job_qw,job_rn,job_sa,job_vv,job_zz,area_a,area_b,area_c,area_d
0,1,-0.607550,-0.664918,-0.208449,9.017895,-0.215764,11.738935,0.854321,0.885827,0.492126,...,0,0,0,0,1,0,0,0,1,0
1,2,0.338053,-0.207748,0.839031,7.766643,0.196869,6.805396,1.394432,0.874016,0.342520,...,0,0,0,0,0,0,0,0,1,0
2,3,-0.184205,1.127906,-0.208449,3.632069,-0.714362,13.671772,-0.495957,0.944882,0.724409,...,0,0,0,0,0,0,0,0,1,0
3,4,-0.526467,-0.440815,-0.208449,5.372942,-0.542432,4.333286,1.124377,0.889764,0.444882,...,0,0,0,0,0,0,0,0,1,0
4,5,-2.851675,1.638861,1.886511,3.822477,-0.473660,5.967121,-2.116291,0.744094,0.661417,...,0,0,0,0,0,0,0,0,0,1


In [5]:
# Convert to numpy - Classification
x_columns = df.columns.drop('product').drop('id')
x = df[x_columns].values
dummies = pd.get_dummies(df['product'])  # Classification
products = dummies.columns
y = dummies.values

In [15]:
x_columns

Index(['income', 'aspect', 'subscriptions', 'dist_healthy', 'save_rate',
       'dist_unhealthy', 'age', 'pop_dense', 'retail_dense', 'crime', 'job_11',
       'job_al', 'job_am', 'job_ax', 'job_bf', 'job_by', 'job_cv', 'job_de',
       'job_dz', 'job_e2', 'job_f8', 'job_gj', 'job_gv', 'job_kd', 'job_ke',
       'job_kl', 'job_kp', 'job_ks', 'job_kw', 'job_mm', 'job_nb', 'job_nn',
       'job_ob', 'job_pe', 'job_po', 'job_pq', 'job_pz', 'job_qp', 'job_qw',
       'job_rn', 'job_sa', 'job_vv', 'job_zz', 'area_a', 'area_b', 'area_c',
       'area_d'],
      dtype='object')

In [16]:
dummies.columns

Index(['a', 'b', 'c', 'd', 'e', 'f', 'g'], dtype='object')

In [17]:
y

array([[False,  True, False, ..., False, False, False],
       [False, False,  True, ..., False, False, False],
       [False,  True, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False,  True, False],
       [False, False,  True, ..., False, False, False],
       [False, False,  True, ..., False, False, False]])

Now that we've preprocessed the data, we can begin the hyperparameter optimization. We start by creating a function that generates the model based on just three parameters. Bayesian optimization works on a vector of numbers, not on a problematic notion like how many layers and neurons are on each layer. To represent this complex neuron structure as a vector, we use several numbers to descirbe this structure.
* **dropout** - The dropout percent for each layer.
* **neuronPct** - What percent of our fixed 5,000 maximum number of neurons do we wish to use? This parameter specifies the total count of neurons in the entire network.
* **nuronShrink** - Neural networks usually start with more neurons on the first hidden layer and then decrease this count for additional layers. This percent specifies how much to shrink subsequent layers based on the previous layer. We stop adding more layers once we run out of neurons (the count specified by neuronPct).

These three numbers define the structure of the neural network. The commends in the below code show exactly how the program constructs the network.

In [6]:
import pandas as pd
import numpy as np
import time
import statistics
from scipy.stats import zscore
from sklearn import metrics
from sklearn.model_selection import StratifiedShuffleSplit
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Convert data to PyTorch tensor
x_tensor = torch.FloatTensor(x).to(device)
y_tensor = torch.LongTensor(np.argmax(y, axis=1)).to(device) # convert one-hot to index

In [7]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, dropout, neuronPct, neuronShrink):
        super(NeuralNetwork, self).__init__()

        layers = []
        neuronCount = int(neuronPct * 5000)
        layer = 0

        prev_count = input_dim
        while neuronCount > 25 and layer < 10:
            layers.append(nn.Linear(prev_count, neuronCount))
            prev_count = neuronCount
            layers.append(nn.PReLU())
            layers.append(nn.Dropout(dropout))
            neuronCount = int(neuronCount * neuronShrink)
            layer += 1

        layers.append(nn.Linear(prev_count, y.shape[1]))
        layers.append(nn.Softmax(dim=1))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

We can test this code to see how it creates a neural network based on three such parameters.

In [8]:
# Create and print the model
model = NeuralNetwork(x.shape[1], 0.2, 0.1, 0.25).to(device)
print(model)

NeuralNetwork(
  (model): Sequential(
    (0): Linear(in_features=47, out_features=500, bias=True)
    (1): PReLU(num_parameters=1)
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=500, out_features=125, bias=True)
    (4): PReLU(num_parameters=1)
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=125, out_features=31, bias=True)
    (7): PReLU(num_parameters=1)
    (8): Dropout(p=0.2, inplace=False)
    (9): Linear(in_features=31, out_features=7, bias=True)
    (10): Softmax(dim=1)
  )
)


We will now create a function to evaluate the neural network using three such parameters. We use bootstrapping because one training run might have "bad luck" with the assigned random weights. We use this function to train and then evaluate the neural network.

In [9]:
# Evaluation function
SPLITS = 2
EPOCHS = 500
PATIENCE = 10

def evaluate_network(learning_rate=1e-3,dropout=0.2,
                        neuronPct=0.1, neuronShrink=0.25):

    boot = StratifiedShuffleSplit(n_splits=SPLITS, test_size=0.1)
    mean_benchmark = []
    epochs_needed = []

    for train, test in boot.split(x, np.argmax(y, axis=1)):
        x_train = x_tensor[train]
        y_train = y_tensor[train]
        x_test = x_tensor[test]
        y_test = y_tensor[test]

        model = NeuralNetwork(x.shape[1],
                        dropout=dropout,
                        neuronPct=neuronPct,
                        neuronShrink=neuronShrink).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        dataset_train = TensorDataset(x_train, y_train)
        loader_train = DataLoader(dataset_train, batch_size=32, shuffle=True)

        best_loss = float('inf')
        patience_counter = 0

        for epoch in range(EPOCHS):
            model.train()
            for batch_x, batch_y in loader_train:
                optimizer.zero_grad()
                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

            model.eval()
            with torch.no_grad():
                outputs_test = model(x_test)
                val_loss = criterion(outputs_test, y_test).item()

            if val_loss < best_loss:
                best_loss = val_loss
                patience_counter = 0
            else:
                patience_counter += 1

            if patience_counter >= PATIENCE:
                epochs_needed.append(epoch)
                break

        # Evaluate
        with torch.no_grad():
            model.eval()
            # Move predictions to CPU for evaluation
            pred = model(x_test).cpu().numpy()
            y_compare = y_test.cpu().numpy()
            score = metrics.log_loss(y_compare, pred)
            mean_benchmark.append(score)

    return -statistics.mean(mean_benchmark)

print(evaluate_network(learning_rate=1e-3,
                       dropout=0.2,
                      neuronPct=0.1,
                      neuronShrink=0.25))

-3.8946162840896514


You can try any combination of our three hyperparameters, plus the learning rate, to see how effective these four numbers are. Of course, our goal is not to manually choose different combinations of these four hyperparamters; we seek to automate.

In [27]:
print(evaluate_network(
    dropout=0.2,
    neuronPct=0.1,
    neuronShrink=0.25))

-4.2907041510118225


Firse, we must install the Bayesian optimization package if we are in Colab.

In [10]:
!pip install bayesian-optimization

We will now automate this process. We define the bounds for each these four hyperparamters and begin the Bayesian optimaization. Once the program finishes, the best combination of hyperparameters found is displayed. The **optimize** function accepts two parameters that will significantly impact how long the process takes to complete:
* **n_iter**: How many steps of Bayesian optimization that you want to perform. The more steps, the more likely you will find a reasonable maximum.
* **init_points**: How many steps of random exploration that you want to perform. Random exploration can help by diversifying the exploration space.

In [11]:
from bayes_opt import BayesianOptimization
import time

# Supress NaN warnings
import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)

# Bounded region of parameter space
pbound = {'dropout': (0.0, 0.499),
          'learning_rate': (0.0, 0.1),
          'neuronPct': (0.01, 0.1),
          'neuronShrink': (0.01, 1)}

optimizer = BayesianOptimization(
    f=evaluate_network,
    pbounds=pbound,
    verbose=2,  # verbose = 1 prints only when a maximum
    # is observed, verbose = 0 is silent
    random_state=1)

start_time = time.time()
optimizer.maximize(init_points=10, n_iter=20)
time_took = time.time() - start_time

print(f"total runtime: {hms_string(time_took)}")
print(optimizer.max )

|   iter    |  target   |  dropout  | learni... | neuronPct | neuron... |
-------------------------------------------------------------------------
| 1         | -8.131    | 0.2081    | 0.07203   | 0.01001   | 0.3093    |
| 2         | -6.393    | 0.07323   | 0.009234  | 0.02676   | 0.3521    |
| 3         | -8.29     | 0.198     | 0.05388   | 0.04773   | 0.6884    |
| 4         | -9.167    | 0.102     | 0.08781   | 0.01246   | 0.6738    |
| 5         | -8.29     | 0.2082    | 0.05587   | 0.02263   | 0.2061    |
| 6         | -9.167    | 0.3996    | 0.09683   | 0.03821   | 0.6954    |
| 7         | -8.29     | 0.4373    | 0.08946   | 0.01765   | 0.04866   |
| 8         | -9.167    | 0.08475   | 0.08781   | 0.01885   | 0.4269    |
| 9         | -11.6     | 0.478     | 0.05332   | 0.07227   | 0.3224    |
| 10        | -9.167    | 0.3426    | 0.08346   | 0.01165   | 0.7526    |
| 11        | -8.346    | 0.07127   | 0.005184  | 0.03199   | 0.3709    |
| 12        | -8.29     | 0.07082   | 

As you can see, the algorithm performed 30 trial iterations. This total iteration count includes ten random and 20 optimization iterations.